# Key Phrase Identification Assignment
https://bcourses.berkeley.edu/courses/1376582/assignments/6650535

Below you will find the code to my key phrase identification algorithm. Section A contains the test code / exerimentation with different methods and section B contains the main algorithm that I am using to detect the key phrases.

In my exerimentation, I tried to various methods starting with the term frequency with unigrams, bigrams, and trigrams. I felt that unigrams and bigrams did a decent job on my collection but, trigrams was a bit lackluster. I first attempted to run this without doing much data cleaning and compared the output of the freqdist after removing punctuation and stopwords and saw that there was only limited improvement. Next, I attempted collocations with the chi-squared method and this also did a decent job with bigrams but not such a great job with trigrams. The method that I spent the most time with was chunking. I began with a simple idea that I wanted to capture any combination of determiner + noun. I iterated on this grammar by checking the output with some changes until I settled on the grammar that you find below which I think does a good job of picking out some key phrases. There is some noise in the output but, overall, this gave me the best results. Finally, I also attempted to see if I could find a more general theme of the text using hyponyms for the most frequently occuring unigrams. However, this proved very diffcult as it was very noisy.

For my final algorithm, I am mainly using my chunking method. I read in the text, break it into sentences, tokenize the sentences (without cleaning, because it helped with accurate identification of proper nouns), run through the chunker, then the freqdist of the chunks, and then, finally, normalize by lower casing and printing out a max of 2000 characters. This is a fairly straight forward method that I think does a decent job of finding the key terms. I think one thing that I can improve on is removing some of the noise from the text but, I didn't want to hurt recall trying to improve precision so, I left it alone.

In [385]:
import nltk
from nltk.corpus import brown, wordnet as wn
from nltk import word_tokenize
from nltk.collocations import *
import string
from collections import defaultdict, OrderedDict

### Helper Functions

In [363]:
def remove_stopwords(wrds):
    stop_words = nltk.corpus.stopwords.words('english')
    return list(filter(lambda w: w.lower() not in stop_words, wrds))

def remove_punctuation(wrds):
    exclude = list(string.punctuation) + ["--","...", "`"]
    return list(filter(lambda w: w[0] not in exclude, wrds))

def remove_digits(wrds):
    exclude = list(string.digits)
    return list(filter(lambda w: w[0] not in exclude, wrds))

def to_tokens(txt):
    pattern = r'''(?x)    # set flag to allow verbose regexps
        <
        | :
        | ([A-Z]\.)+        # abbreviations, e.g. U.S.A.
       | \w+([-']\w+)*        # words with optional internal hyphens
       | \$?\d+(\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
       | \.\.\.            # ellipsis
       | [.,;"'?():-_`]+  # these are separate tokens
     '''
    return nltk.regexp_tokenize(txt, pattern)

def to_sents(txt):
    sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
    return sent_tokenizer.tokenize(txt)

## A) Testing

I tried various methods that we covered in class below individually to see how well, I could perform just using each single method. I wanted to use this to guide how I approached my final algorithm.

In [364]:
tal_text = open("../tal_stories/tal_text_clean.txt")
txt = tal_text.read()
tal_text.close()
tokens = to_tokens(txt)
tokens = remove_stopwords(tokens)
tokens = remove_punctuation(tokens)
filtered_tokens = list(filter(lambda x: len(x) > 5, tokens))
# filtered_tokens = map(lambda x: x.lower(), filtered_tokens)

### 1. Frequent Terms

In [365]:
def run_freq_terms(input_tokens, n):
    fq = nltk.FreqDist(input_tokens)  # compute the frequency distribution
    return fq.most_common(n)

print("\n Most Common Unigrams")
print(run_freq_terms(filtered_tokens, 50))

# bigrams
bgs = nltk.bigrams(tokens)
print("\n Most Common Bigrams")
print(run_freq_terms(bgs, 50))

# trigrams
bgs = nltk.trigrams(tokens)
print("\n Most Common Trigrams")
print(run_freq_terms(bgs, 50))


 Most Common Unigrams
[('people', 300), ('patent', 227), ('really', 149), ('Ventures', 120), ('Intellectual', 120), ('actually', 118), ("didn't", 111), ('company', 111), ('patents', 109), ('something', 106), ('things', 105), ('called', 102), ('companies', 101), ("that's", 101), ("you're", 99), ('little', 90), ("That's", 83), ('number', 81), ("they're", 80), ('started', 77), ('Anthony', 68), ('trying', 68), ('around', 67), ("there's", 65), ('anything', 63), ('business', 63), ('different', 61), ('someone', 60), ('thought', 59), ("doesn't", 58), ('internet', 56), ('getting', 55), ('program', 55), ('saying', 54), ("There's", 52), ('police', 51), ('problem', 50), ('wanted', 50), ('working', 49), ('numbers', 49), ('making', 49), ('cartel', 48), ("wasn't", 47), ('software', 47), ('another', 46), ("They're", 45), ('system', 45), ('somebody', 44), ("wouldn't", 44), ('probably', 44)]

 Most Common Bigrams
[(('Intellectual', 'Ventures'), 120), (("don't", 'know'), 58), (('Chris', 'Crawford'), 42)

### 2. Collocations

In [366]:
def run_collocations(input_tokens, collocation_type):
    
    if collocation_type == 'bigram':
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        finder = BigramCollocationFinder.from_words(input_tokens, 5)
        finder.apply_freq_filter(5)

        print('Printing Top 100 Collocations')
        for item in finder.nbest(bigram_measures.chi_sq, 100):
            print(item)

    elif collocation_type == 'trigram':
        trigram_measures = nltk.collocations.TrigramAssocMeasures()
        finder = TrigramCollocationFinder.from_words(input_tokens, 7)
        finder.apply_freq_filter(5)

        print('Printing Top 100 Collocations')
        for item in finder.nbest(trigram_measures.chi_sq, 100):
            print(item)
    
    else:
        print('You must choose either bigram or trigram')

### 3. Information obtained from Syntax aka Partial Parsing (Chunking)

In [367]:
grammar = r"""
  NP: {(<DT|PP\$>?<JJ>?<NN.>)*}
      {<NNP>+}
"""
cp = nltk.RegexpParser(grammar)
sentences = to_sents(txt)
sentences = remove_punctuation(sentences)
interesting_chunks = []

for sentence in sentences[1:100]:
    tree = cp.parse(nltk.pos_tag(to_tokens(sentence)))
    for t in tree.subtrees():
        if t.label() == 'NP':  
            print(t.leaves())

[('podcasts', 'NNS')]
[('some', 'DT'), ('interviews', 'NNS')]
[('colleagues', 'NNS')]
[('Zoe', 'NNP'), ('Chace', 'NNP')]
[("She's", 'NNS')]
[('the', 'DT'), ('reporters', 'NNS')]
[('Planet', 'NNP'), ('Money', 'NNP')]
[('Zoe', 'NNP')]
[('guys', 'NNS')]
[('Jim', 'NNP'), ('Logan', 'NNP')]
[('Richard', 'NNP'), ('Baker', 'NNP')]
[('Personal', 'NNP'), ('Audio', 'NNP')]
[('podcasts', 'NNS')]
[('Zoe', 'NNP')]
[('the', 'DT'), ('legalities', 'NNS')]
[('Jim', 'NNP'), ('Logan', 'NNP')]
[('the', 'DT'), ('inventors', 'NNS')]
[("Here's", 'NNS')]
[('Personal', 'NNP'), ('Audio', 'NNP')]
[('the', 'DT'), ('iPod', 'NNP')]
[("it's", 'NNP')]
[('homes', 'NNS')]
[('an', 'DT'), ('MP3', 'NNP')]
[('all', 'DT'), ('kinds', 'NNS')]
[('Jim', 'NNP')]
[('these', 'DT'), ('words', 'NNS')]
[('ideas', 'NNS')]
[('newfangled', 'JJ'), ('MP3', 'NNP')]
[('articles', 'NNS')]
[('tapes', 'NNS')]
[('first', 'JJ'), ('podcasts', 'NNS')]
[('tapes', 'NNS')]
[('These', 'DT'), ('guys', 'NNS')]
[('Welcome', 'NNP')]
[('"A', 'NNP')]
[('the'

### 4. Using Semantic Similarity / Finding Higher-Level Concepts

In [368]:
## Attempt to find most common hyponyms for the given set of terms
def categories_from_hypernyms(termlist, num_cats=20):
    
    hypterms = []
    for term in termlist:                  # for each term
        s = wn.synsets(term.lower(), 'n')  # get its nominal synsets
        for syn in s:                      # for each lemma synset
            hypterms += syn.hyponyms()
                
    hypfd = nltk.FreqDist(hypterms)
    print(hypfd.most_common(30))

In [369]:
categories_from_hypernyms(single_word_chunks)

[(Synset('homebound.n.01'), 215), (Synset('living.n.02'), 215), (Synset('poor_people.n.01'), 215), (Synset('disabled.n.01'), 215), (Synset('nationality.n.01'), 215), (Synset('ionian.n.02'), 215), (Synset('populace.n.01'), 215), (Synset('laity.n.01'), 215), (Synset('ancients.n.01'), 215), (Synset('dorian.n.02'), 215), (Synset('migration.n.02'), 215), (Synset('damned.n.01'), 215), (Synset('rank_and_file.n.02'), 215), (Synset('achaean.n.02'), 215), (Synset('nation.n.02'), 215), (Synset('business_people.n.01'), 215), (Synset('unconfessed.n.01'), 215), (Synset('chosen_people.n.01'), 215), (Synset('governed.n.01'), 215), (Synset('timid.n.01'), 215), (Synset('retreated.n.01'), 215), (Synset('uninitiate.n.01'), 215), (Synset('wounded.n.01'), 215), (Synset('dead.n.01'), 215), (Synset('smart_money.n.03'), 215), (Synset('population.n.01'), 215), (Synset('defeated.n.01'), 215), (Synset('episcopacy.n.01'), 215), (Synset('network_army.n.01'), 215), (Synset('enemy.n.03'), 215)]


## B) The Algorithm

In [413]:
def parse_tree(tree, interesting_chunks):
    try:
        tree.label()
    except AttributeError:
        return
    else:
        if tree.label() == 'NP':  
            interesting_chunks.append(tree.leaves())
        else:
            for child in tree:
                parse_tree(child, interesting_chunks)

def algo(collection, max_output_len=2000):    
    """ This implementation of the algorithm accepts 
        both raw text or list of sentences.
    """
    sentences = []
    if isinstance(collection, str):
        sentences = to_sents(collection)
    elif isinstance(collection, list):
        sentences = collection
    else:
        print("collection passed to function must \
              be either a list of sentences or a raw text.")

    grammar = r"""
      NP: {(<DT|PP\$>?<JJ>?<NN.>)*}
          {<NNP>+}
    """
    cp = nltk.RegexpParser(grammar)
    
    interesting_chunks = []
    for sentence in sentences:
        tree = cp.parse(nltk.pos_tag(to_tokens(sentence)))
        parse_tree(tree, interesting_chunks)
    
    chunks = []
    
    for chunk in interesting_chunks:
        # Ignore single word chunks for now
        if len(chunk) > 1:
            chunks.append(" ".join([sent[0] for sent in chunk]))
            
    freq_terms = run_freq_terms(chunks, 150)
    
    output_len = 0
    output = OrderedDict()
    for term in freq_terms:
        output_len += len(term[0])
        if output_len > max_output_len:
            break
        else:
            output[term[0].lower()] = "don't matter"
    
    return output.keys()

## Output

### Personal Collection

In [391]:
tal_text = open("../tal_stories/tal_text_clean.txt")
txt = tal_text.read()
tal_text.close()
output = algo(txt)
for line in output:
    print(line)

intellectual ventures
chris crawford
public radio international
chris crawford's
oasis research
ira glass
this american life
the people
peter detkin
nathan myhrvold
other words
silicon valley
mike daisey
other people
alex blumberg
these people
adrian schoolcraft
the patents
the numbers
tom ewing
these patents
graham rayman
the scrambler
the cali
julian vinegas
the experiments
jack byrd's
the agents
san francisco
jack byrd
all kinds
the cells
wbez chicago
the dea
new york
it's this american life
the workers
kwon holdings
the girls
many people
"i don't
seth lind
chuck campos
julie snyder
the companies
nancy updike
prime numbers
the boys
chicago public radio
laura sydell
michael smith
senate bill
the details
fight club
ben calhoun
chris sacca
the officers
many times
these pictures
the ones
these guys
sarah koenig
the streets
no employees
the things
those patents
the guards
the machines
jonathan menjivar
ian spaulding
those things
alissa shipp
internal affairs
the data
robyn semien
mary ar

### Brown News

In [392]:
brown = []
for sent in nltk.corpus.brown.sents(categories="news"):
    brown.append(" ".join(sent))

output = algo(brown)
for line in output:
    print(line)

the united
the president
the congo
the u.s.
president kennedy
the house
the white house
the senate
the people
the legislature
new york
the yankees
the university
premier khrushchev
the anti-trust laws
the u. s.
the soviet
san francisco
the u.n.
the american
the rev
los angeles
new orleans
the kennedy
the sales
the children
el paso
ap )
the members
new jersey
kansas city
american catholic
the soviet union
the communists
the rules committee
the president's
the army
the birds
the hughes
the students
the defendants
hong kong
arnold palmer
the years
the orioles
the dreadnought
last saturday
the beardens
emory university
the masters
the republican
white house
the problems
the bible
the anti-monopoly laws
new york city
president-elect kennedy
dental schools
the puppets
dallas county
the laws
the teamsters
the st
the children's
the government
the toll-road bonds
mickey mantle
east providence
rhode island
the kremlin
the state's
pennsylvania avenue
the schools
capitol hill
southeast asia
the da

### Mystery

In [442]:
from urllib.request import urlopen, urlretrieve

# Mystery Text URL goes here
# url = ""

# page_text = []
# urlretrieve(url, "mystery_expository.txt")

mystery_text = open("mystery_expository.txt")
txt = mystery_text.read()
mystery_text.close()

output = algo(txt)

for line in output:
    print(line)

the characters
the special effects
the "
special effects
the movie's
the movies
the musketeers
the dogs
the filmmakers
those films
the girls
the producers
the bugs
the kids
the scenes
the games
bad guys
the film's
signs wonders
the others
the actors
the writers
some movies
the performances
many characters
many times
dramatic contrivances
the lambs
the streets
sports films
popular kids
the films
the words
the replacements
no points
a sports
the fights
the english
the authorities
the apes
the theaters
these people
the cops
different perspectives
the jokes
big john's
these moments
many things
nervous smiles
the bowels
the adventures
the early days
shelton's films
a sci-fi
the piano's braces
the mortals
the ramones
the editors
the secrets
automatic weapons
life's riddles
who's films
sonny chiba's
own parallels
hot reviews
these things
the shelves
unexpected events
other things
various ideas
bird's-eye-view shots
the residents
ryan tries
the few people
some martial arts
physical attributes


In [441]:
from urllib.request import urlopen, urlretrieve

# Mystery Text URL goes here
# url = ""

# page_text = []
# urlretrieve(url, "mystery_expository.txt")

mystery_text = open("mystery_text_narrative.txt")
txt = mystery_text.read()
mystery_text.close()

output = algo(txt)

for line in output:
    print(line)

shere khan
kala nag
the pack
little toomai
father wolf
the monkeys
petersen sahib
the jungle
the elephants
gray brother
mother wolf
the law
big toomai
the men
the council rock
the buffaloes
the wolves
the branches
kala nag's
"y es
the terms
the trees
the bullocks
sea catch
the seals
the walls
the bulls
the hills
shere khan's
the project gutenberg-tm
the cubs
machua appa
project gutenberg-tm
the united
little brother
the waingunga
project gutenberg-tm electronic works
the project gutenberg literary archive foundation
sea vitch
the council
the beaches
the free people
the lines
the others
the villagers
the guns
the keddah
petersen sahib's
the cold lairs
the black panther
the eyes
the red flower
the sides
sea cow
the beasts
"t hat's
the pacific
a project gutenberg-tm
the plains
the bear
white men
the tree-tops
electronic works
the ways
the amir
the jungles
the young wolves
the cows
the viceroy
mang the bat
the people
shiva the preserver
the jungle ."
the foundation
the panther
the marks
th